Step 1 Spliting Historical and upcoming datas

In [1]:
import pandas as pd

file_path = "data/TelcoCustomerChurnData.csv"
df = pd.read_csv(file_path)

df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
print(df["TotalCharges"].isnull().sum())#initially 11 rows were NaN
df = df.dropna(subset=["TotalCharges"])
df = df.drop(columns=["customerID"])
df["Churn"] = df["Churn"].map({"Yes":1,"No":0})

11


In [2]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [3]:
df = df.reset_index(drop=True)
df["record_time"] = df.index

df = df.sort_values(by="record_time").reset_index(drop=True)

In [4]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,record_time
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,No,No,No,One year,No,Mailed check,56.95,1889.50,0,1
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,2
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,3
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,4


In [5]:
split_index = int(len(df)*0.75)

ref_data = df.iloc[:split_index].copy()
prod_data = df.iloc[split_index:].copy()

print("Reff data:",ref_data.shape)
print("Prod data:",prod_data.shape)

Reff data: (5274, 21)
Prod data: (1758, 21)


In [6]:
ref_data = ref_data.drop(columns=["record_time"])
prod_data = prod_data.drop(columns=["record_time"])

In [7]:
ref_data.to_csv("data/train_data.csv",index=False)
prod_data.to_csv("data/new_batch.csv",index=False)

Step 2 
Introducing Drifts

In [8]:
print("Reference MonthlyCharges Summary:")
ref_data["MonthlyCharges"].describe()

Reference MonthlyCharges Summary:


count    5274.000000
mean       64.942937
std        30.161055
min        18.250000
25%        35.612500
50%        70.475000
75%        90.050000
max       118.750000
Name: MonthlyCharges, dtype: float64

In [9]:
print("Production MonthlyCharges Summary:")
prod_data["MonthlyCharges"].describe()
#done to justify that before injection both look similar....

Production MonthlyCharges Summary:


count    1758.000000
mean       64.364022
std        29.863928
min        18.550000
25%        35.525000
50%        70.150000
75%        89.437500
max       118.200000
Name: MonthlyCharges, dtype: float64

In [10]:
#now we inject changes in production data
prod_data["MonthlyCharges"] = prod_data["MonthlyCharges"]*1.25
prod_data.loc[prod_data["Contract"]=="Month-to-month","Contract"] = "Two year"
prod_data.loc[prod_data["InternetService"]=="DSL","InternetService"] = "Fiber optic"

In [11]:
print("\nAFTER DRIFT")

print("\nReference - MonthlyCharges summary:")
print(ref_data["MonthlyCharges"].describe())

print("\nProduction - MonthlyCharges summary:")
print(prod_data["MonthlyCharges"].describe())

print("\nReference - Contract distribution:")
print(ref_data["Contract"].value_counts())

print("\nProduction - Contract distribution:")
print(prod_data["Contract"].value_counts())

print("\nReference - InternetService distribution:")
print(ref_data["InternetService"].value_counts())

print("\nProduction - InternetService distribution:")
print(prod_data["InternetService"].value_counts())


AFTER DRIFT

Reference - MonthlyCharges summary:
count    5274.000000
mean       64.942937
std        30.161055
min        18.250000
25%        35.612500
50%        70.475000
75%        90.050000
max       118.750000
Name: MonthlyCharges, dtype: float64

Production - MonthlyCharges summary:
count    1758.000000
mean       80.455027
std        37.329910
min        23.187500
25%        44.406250
50%        87.687500
75%       111.796875
max       147.750000
Name: MonthlyCharges, dtype: float64

Reference - Contract distribution:
Contract
Month-to-month    2927
Two year          1249
One year          1098
Name: count, dtype: int64

Production - Contract distribution:
Contract
Two year    1384
One year     374
Name: count, dtype: int64

Reference - InternetService distribution:
InternetService
Fiber optic    2326
DSL            1800
No             1148
Name: count, dtype: int64

Production - InternetService distribution:
InternetService
Fiber optic    1386
No              372
Name: count

In [12]:
prod_data.to_csv("data/new_batch_drifted.csv",index=False)

NOTE: now the prod_data shows the new_batch_drifted data, be aware of that

In [13]:
numerical_cols = ref_data.select_dtypes(include=["int64","float64"]).columns
categorical_cols = ref_data.select_dtypes(include=["object"]).columns
print("\nNumerical_Columns:\n",numerical_cols)
print("\nCategorical_Columns:\n",categorical_cols)


Numerical_Columns:
 Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'Churn'], dtype='object')

Categorical_Columns:
 Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object')


Importing statistical tests

In [14]:
from scipy.stats import ks_2samp, chi2_contingency

In [15]:
numerical_drift_results = {}

for col in numerical_cols:
    stat,p_value = ks_2samp(ref_data[col],prod_data[col])
    numerical_drift_results[col] = {
        "ks_statistic" : stat,
        "p_value" : p_value,
        "drift_detected" : p_value<0.05
    }
numerical_drift_df = pd.DataFrame(numerical_drift_results).T
numerical_drift_df

,ks_statistic,p_value,drift_detected
SeniorCitizen,0.002655,1.0,False
tenure,0.019151,0.715207,False
MonthlyCharges,0.247819,0.0,True
TotalCharges,0.016875,0.843844,False
Churn,0.014221,0.950803,False


In [16]:
categorical_drift_results = {}

for col in categorical_cols:
    ref_counts = ref_data[col].value_counts()
    prod_counts = prod_data[col].value_counts()

    combined = pd.concat([ref_counts,prod_counts],axis=1,sort=True).fillna(0)
    combined.columns = ["reference","production"]

    chi2,p_value,_,_ = chi2_contingency(combined)

    categorical_drift_results[col] = {
        "chi2_statistic": chi2,
        "p_value": p_value,
        "drift_detected": p_value < 0.05
    }

categorical_drift_df = pd.DataFrame(categorical_drift_results).T
categorical_drift_df

,chi2_statistic,p_value,drift_detected
gender,0.022945,0.879601,False
Partner,0.118651,0.730502,False
Dependents,1.01621,0.31342,False
PhoneService,4.013609,0.045134,True
MultipleLines,5.906142,0.052179,False
InternetService,901.609619,0.0,True
OnlineSecurity,0.377649,0.827932,False
OnlineBackup,0.588495,0.745092,False
DeviceProtection,1.560962,0.458186,False
TechSupport,0.992153,0.608915,False
